In [125]:
##Brute-Force, Exhaustive Search!
import pandas as pd
#import statsmodels.formula.api as smf
import numpy as np
import scipy.stats
#from scipy.stats import norm
from sklearn.linear_model import LinearRegression  
import itertools
import copy
import warnings
import time
import pickle
with open('stringlist.pkl', 'rb') as f:
      stringlist = pickle.load(f)

In [3]:
## Get test Data as a Pandas Dataframe! Make sure to have numerical data only! (pd.get_dummies might help)

file ="/Users/geoffreykasenbacher/Desktop/projects/branch and bound/Branch-and-Bound/Datasets/nuclear.csv"

data1 = pd.read_csv(file,sep=",").astype(float) #.drop(data.columns[1], axis=1)
data = data1.drop(data1.columns[0], axis=1)

Y = list(data)[4]
#X = list(data)[1:]

print(data)

      cost   date    t1    t2     cap   pr   ne   ct   bw  cum.n   pt
0   460.05  68.58  14.0  46.0   687.0  0.0  1.0  0.0  0.0   14.0  0.0
1   452.99  67.33  10.0  73.0  1065.0  0.0  0.0  1.0  0.0    1.0  0.0
2   443.22  67.33  10.0  85.0  1065.0  1.0  0.0  1.0  0.0    1.0  0.0
3   652.32  68.00  11.0  67.0  1065.0  0.0  1.0  1.0  0.0   12.0  0.0
4   642.23  68.00  11.0  78.0  1065.0  1.0  1.0  1.0  0.0   12.0  0.0
5   345.39  67.92  13.0  51.0   514.0  0.0  1.0  1.0  0.0    3.0  0.0
6   272.37  68.17  12.0  50.0   822.0  0.0  0.0  0.0  0.0    5.0  0.0
7   317.21  68.42  14.0  59.0   457.0  0.0  0.0  0.0  0.0    1.0  0.0
8   457.12  68.42  15.0  55.0   822.0  1.0  0.0  0.0  0.0    5.0  0.0
9   690.19  68.33  12.0  71.0   792.0  0.0  1.0  1.0  1.0    2.0  0.0
10  350.63  68.58  12.0  64.0   560.0  0.0  0.0  0.0  0.0    3.0  0.0
11  402.59  68.75  13.0  47.0   790.0  0.0  1.0  0.0  0.0    6.0  0.0
12  412.18  68.42  15.0  62.0   530.0  0.0  0.0  1.0  0.0    2.0  0.0
13  495.58  68.92  1

In [4]:
## exhaustive search - Brute Force. create class here

class BruteForce: 
    
    def __init__(self, data, Y, lowerbound, upperbound, adjusted_R2):
        self.data = data
        self.Y = Y
        self.lowerbound = lowerbound
        self.upperbound = upperbound
        self.adjusted_R2 = adjusted_R2
        
        datalist = list(self.data)
        self.X = [x for x in datalist if x != Y]
        
    def subset(self, X):
        _set = self.powerset(X)
        subset = []
        for i in _set:
            if self.upperbound >= len(i) >= self.lowerbound:
                subset.append(i)
        return subset
   
    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                #print(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(data[X],data[Y])  
        
        #returns R^2
        r2 = regressor.score(data[X], data[Y])
        
        # Calculates adjusted R^2
        n = len(data)
        k = len(self.X)
        adj_r2 = ( 1 - (( (1-r2)*(n-1) )/ (n - k -1)) )
        
        if self.adjusted_R2 == True:
            return adj_r2
        elif self.adjusted_R2 == False:
            return r2
        else:
            raise ValueError("adjusted_r2 needs to be boolean: True or False (ノಠ益ಠ)ノ")
    
    def search(self):
        solutions = []
        #combos = self.powerset(self.X)
        combos = self.subset(self.X)
        for i in combos:
            solutions.append(self.regress(i, self.Y, self.data))  
        return solutions, combos
    
    def solve(self):
        sol, combos = self.search()
        highest_R2 = combos[np.argmax(sol)]
        R2 = max(sol)
        
        print("higehst R^2 is", R2, "with dependant variables:", highest_R2)
        return highest_R2, R2     

In [13]:
t, r2 = BruteForce(data, Y = 'date', lowerbound = 2, upperbound = 5, adjusted_R2 = True).solve()

t = (list(data))



higehst R^2 is 0.8194028439321454 with dependant variables: ['cost', 't1', 'ne', 'cum.n', 'pt']


In [6]:
## for BruteForce add exceptions, warnings and maybe class inheritance! boolean check for length of bounds

In [134]:
## this will solve all possible linear regressions and return the one with highest R^2

class ExhaustiveSearch: 
    
    def __init__(self, data, lowerbound, upperbound, adjusted_R2):
        self.data = data
        self.lowerbound = lowerbound
        self.upperbound = upperbound
        self.adjusted_R2 = adjusted_R2
        
        ## Raising Expections and Warnings here
        if len(list(self.data)) > 15:
            warnings.warn(stringlist[0], Warning)
        if str(type(self.data))  != "<class 'pandas.core.frame.DataFrame'>":
            raise TypeError("Input data must be of type pandas Dataframe ( ་ ⍸ ་ )") 
        if type(self.lowerbound) and type(self.upperbound) != int:
            raise TypeError("Bounds must be integers! ಠ_๏") 
    
    def subset(self, X):
        _set = self.powerset(X)
        subset = []
        for i in _set:
            if self.upperbound >= len(i) >= self.lowerbound:
                subset.append(i)
        return subset
        
    def assign(self):
        XX = list(self.data)
        dic = {}
        for i in XX:
            Y = i
            #X = self.powerset([x for x in XX if x != Y])
            X = self.subset([x for x in XX if x != Y])
            dic[Y] = X
        return dic

    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(self.data[X],self.data[Y])
        
        #returns R^2
        r2 = regressor.score(self.data[X], self.data[Y])
        
        #returns Adjusted R^2 
        n = len(self.data)
        k = len(list(self.data[X]))
        adj_r2 = (1 - ( ((1 - r2)*(n - 1))/(n - k - 1) ))
        
        if self.adjusted_R2 == True:
            return adj_r2
        elif self.adjusted_R2 == False:
            return r2
        else:
            raise ValueError("adjusted_r2 needs to be boolean: True or False (ノಠ益ಠ)ノ")
    
    def search(self):
        dic = self.assign()
        newdic = {}
        for y in dic:
            for x in dic[y]:
                newdic[self.regress(x, y, data)] = y, x
        new={}
        sd = sorted(newdic.items(), reverse=True)
        for k,v in sd:
            new[k] = v
        return new
    
    def solve(self):
        results = self.search()
        highest_R2 = list(results.keys())[0]
        best_vars = results[highest_R2]
        
        print("highest R^2 is", highest_R2, "with dependant/indepedant variables:", best_vars)
        return results

In [135]:
wow = ExhaustiveSearch(data = data, lowerbound = 2, upperbound = 2, adjusted_R2 = True).solve()

highest R^2 is 0.7823640370462159 with dependant/indepedant variables: ('date', ['cost', 't1'])


In [31]:
## want to format the output properly and maybe add some funny string art ;-), also maybe store the strings in a file? 

In [87]:
giraffe="""
            
            
            Dear fellow Caveman, surely you are aware that your dataset exceeds 15 variables so it may take
            a looong time to compute if no bounds are specified, consider setting reasonable bounds
            to find a quick solution, happy rampage!
            
                                   ._ o o
                                   \_`-)|_
                                ,""       \ 
                              ,"  ## |   ಠ ಠ. 
                            ," ##   ,-\__    `.
                          ,"       /     `--._;)
                        ,"     ## /
                      ,"   ##    /

            """

In [60]:
yoda ='''

                _.' :  `._                                            
            .-.'`.  ;   .'`.-.                                        
   __      / : ___\ ;  /___ ; \      __                               
 ,'_ ""--.:__;".-.";: :".-.":__;.--"" _`,                             
 :' `.t""--.. '<@.`;_  ',@>` ..--""j.' `;                             
      `:-.._J '-.-'L__ `-- ' L_..-;'                                  
        "-.__ ;  .-"  "-.  : __.-"       Oh! Aware you are surely: variables in dataset exceed 10.     
            L ' /.------.\ ' J           Chose wisely you must the bounds.       
             "-.   "--"   .-"            Do or do not.     
            __.l"-:_JL_;-";.__           There is no try.   
         .-j/'.;  ;""""  / .'\"-.                                     
       .' /:`. "-.:     .-" .';  `.       
    .-"  / ;  "-. "-..-" .-"  :    "-.     
 .+"-.  : :      "-.__.-"      ;-._   \                               
 ; \  `.; ;                    : : "+. ;                              
 :  ;   ; ;                    : ;  : \:                              
 ;  :   ; :                    ;:   ;  :                              
: \  ;  :  ;                  : ;  /  ::                              
;  ; :   ; :                  ;   :   ;:                              
:  :  ;  :  ;                : :  ;  : ;                              
;\    :   ; :                ; ;     ; ;                              
: `."-;   :  ;              :  ;    /  ;                              
 ;    -:   ; :              ;  : .-"   :                              
 :\     \  :  ;            : \.-"      :                              
  ;`.    \  ; :            ;.'_..--  / ;                              
  :  "-.  "-:  ;          :/."      .'  :                             
   \         \ :          ;/  __        :                             
    \       .-`.\        /t-""  ":-+.   :                             
     `.  .-"    `l    __/ /`. :  ; ; \  ;                             
       \   .-" .-"-.-"  .' .'j \  /   ;/                              
        \ / .-"   /.     .'.' ;_:'    ;                               
         :-""-.`./-.'     /    `.___.'                                
               \ `t  ._  /                                         
                "-.t-._:'                                             

'''

In [61]:
print(yoda)



                _.' :  `._                                            
            .-.'`.  ;   .'`.-.                                        
   __      / : ___\ ;  /___ ; \      __                               
 ,'_ ""--.:__;".-.";: :".-.":__;.--"" _`,                             
 :' `.t""--.. '<@.`;_  ',@>` ..--""j.' `;                             
      `:-.._J '-.-'L__ `-- ' L_..-;'                                  
        "-.__ ;  .-"  "-.  : __.-"       Oh! Aware you are surely: variables in dataset exceed 10.     
            L ' /.------.\ ' J           Chose wisely you must the bounds.       
             "-.   "--"   .-"            Do or do not.     
            __.l"-:_JL_;-";.__           There is no try.   
         .-j/'.;  ;""""  / .'"-.                                     
       .' /:`. "-.:     .-" .';  `.       
    .-"  / ;  "-. "-..-" .-"  :    "-.     
 .+"-.  : :      "-.__.-"      ;-._   \                               
 ; \  `.; ;                    : : "+

In [118]:
c3po="""

  I'm programmed for etiquette, not destruction.                                         
  You may want to consider setting appropriate bounds to                                           
  find a solution in a reasoable time!
  
               \  .-.                                
                 /_ _\                               
                 |o^o|                               
                 \ _ /                               
                .-'-'-.              BLEEP BOOP BLOOP
              /`)  .  (`\            (THIS MEANS YES)
             / /|.-'-.|\ \         /                 
             \ \| (_) |/ /  .-""-.                   
              \_\'-.-'/_/   /[] _ _\                  
              /_/ \_/ \_\ _|_o_LII|_                 
                |'._.'|  / | ==== | \                
                |  |  |  |_| ==== |_|                
                 \_|_/    ||" ||  ||                 
                 |-|-|    ||LI  o ||                 
                 |_|_|    ||'----'||                 
                /_/ \_\  /__|    |__\                
"""

dataerror ='''

Input data must be of type pandas Dataframe ( ་ ⍸ ་ )
'''

bounderror ='''

"Bounds must be integers! ಠ_ಠ'''

valueerrorstring ='''

adjusted_r2 needs to be boolean: True or False (ノಠ益ಠ)ノ
'''

In [129]:
stringlist = []
stringlist.append(c3po)
stringlist.append(dataerror)
stringlist.append(bounderror)
stringlist.append(valueerrorstring)
stringlist.append(yoda)

In [131]:
print(stringlist[4])



                _.' :  `._                                            
            .-.'`.  ;   .'`.-.                                        
   __      / : ___\ ;  /___ ; \      __                               
 ,'_ ""--.:__;".-.";: :".-.":__;.--"" _`,                             
 :' `.t""--.. '<@.`;_  ',@>` ..--""j.' `;                             
      `:-.._J '-.-'L__ `-- ' L_..-;'                                  
        "-.__ ;  .-"  "-.  : __.-"       Oh! Aware you are surely: variables in dataset exceed 10.     
            L ' /.------.\ ' J           Chose wisely you must the bounds.       
             "-.   "--"   .-"            Do or do not.     
            __.l"-:_JL_;-";.__           There is no try.   
         .-j/'.;  ;""""  / .'"-.                                     
       .' /:`. "-.:     .-" .';  `.       
    .-"  / ;  "-. "-..-" .-"  :    "-.     
 .+"-.  : :      "-.__.-"      ;-._   \                               
 ; \  `.; ;                    : : "+

In [132]:
print(len(stringlist))

5


In [133]:
with open('stringlist.pkl', 'wb') as f:
       pickle.dump(stringlist, f)

In [94]:
'''with open('stringlist.pkl', 'rb') as f:
      stringlist = pickle.load(f)'''